In [37]:
import pandas as pd
import numpy as np

In [38]:
data = pd.read_csv('new_data/complete-data.csv')

data.sort_values('year')

,playerID,year,tmID,GP,oRebounds,dRebounds,dq,ft%,fg%,three%,...,award_count,career_year,playoff_progression_rolling,playoff_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,rpg_rolling,ppg_rolling,stats_post_rolling
691,397,3,573,24.0,1.38,1.83,0.00,0.61,0.41,0.00,...,0.0,3,0.5,0.5,-112.170,0.0,24.65,2.605,5.545,-0.285
287,191,3,568,3.0,0.33,0.33,0.00,0.75,0.50,0.00,...,0.0,3,1.5,1.0,-100.605,0.0,30.65,3.235,6.050,-0.370
97,70,3,559,32.0,1.44,4.38,0.00,0.83,0.46,0.40,...,0.0,3,0.5,0.5,-212.575,0.0,79.70,6.470,9.690,0.215
284,187,3,573,32.0,0.50,1.50,0.00,0.76,0.38,0.32,...,0.0,3,0.0,0.0,-171.655,0.0,22.05,2.215,7.270,0.000
900,504,3,558,32.0,0.97,3.98,0.05,1.70,0.71,0.83,...,0.0,3,2.5,1.0,-101.850,0.0,33.90,2.225,3.500,-0.115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,247,11,556,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,9,0.5,0.5,-267.790,0.0,53.75,6.000,16.850,-0.135
424,260,11,574,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,6,1.0,0.5,-228.340,0.0,54.55,5.355,9.310,-0.205
427,262,11,745,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,5,0.5,0.5,-131.025,0.0,20.25,2.445,4.365,-0.215
836,466,11,573,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,9,4.0,1.0,-173.405,0.0,46.45,4.360,14.360,-1.135


In [39]:
with open('stats.txt', 'w') as file:
    for col in data.columns:
        if type(data[col][0]) == str: 
            continue
        file.write("-"*50 + "\n")
        file.write(f"Statistics of {col}\n")
        file.write("-"*50 + "\n")
        # data[col].plot.box();  # You can enable this if you want to create a plot

        q3, q1 = np.percentile(data[col], [75 ,25])

        file.write(f"Min: {data[col].min()}\n")
        file.write(f"Max: {data[col].max()}\n")
        file.write(f"Mean: {data[col].mean()}\n")
        file.write(f"Median: {data[col].median()}\n")
        file.write(f"Standard deviation: {data[col].std()}\n")
        file.write(f"Skewness: {data[col].skew()}\n")
        file.write(f"Variance: {data[col].var()}\n")
        file.write(f"Kurtosis: {data[col].kurtosis()}\n")
        file.write(f"Interquartile Range: {q3 - q1} | Q1: {q1} | Q3: {q3}\n")

In [40]:
outliers_std = {}
all_outliers = []  # Array to store all outlier rows

#columns to skip
not_skip_cols = ['height', 'weight']

for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    data_mean, data_std = data[col].mean(), data[col].std()
    # identify outliers
    cut_off = data_std * 4
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = pd.concat([data.loc[data[col] < lower], data.loc[data[col] > upper]])
    outliers_std[col] = outliers
    all_outliers.extend(outliers.values.tolist())  # Add outliers to the array

for key in outliers_std.keys():
    if len(outliers_std[key]) > 0:
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_std[key][key])

# Printing all outlier rows
print("All outlier rows:")
for outlier in all_outliers:
    print(outlier)

All outlier rows:


In [41]:
outliers_ids_std = set([val for key in outliers_std.keys() for val in outliers_std[key]['playerID']])
outlier_ratio = len(outliers_ids_std) / len(data)
outlier_ratio

0.0

In [42]:
outliers_z = {}
for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    z_val = (data[col] - data[col].mean()) / data[col].std()
    outliers_z[col] = data.loc[(z_val < -4) | (z_val > 4)]

for key in outliers_z.keys():
    if (len(outliers_z[key]) > 0):
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_z[key][key])

In [43]:
outliers_ids_z = set([val for key in outliers_z.keys() for val in outliers_z[key]['playerID']])
outlier_ratio = len(outliers_ids_z) / len(data)
outlier_ratio

0.0

In [44]:
outliers_ids = outliers_ids_std.union(outliers_ids_z)
print(f"Total outliers: {len(outliers_ids)}")
print(f"Total outliers ratio: {len(outliers_ids) / len(data)}")
print(outliers_ids)
#create a new df with outliers removed
manual_outlier_df = data[~data['playerID'].isin(outliers_ids)]

manual_outlier_df.duplicated(subset=['playerID', 'year']).sum()

Total outliers: 0
Total outliers ratio: 0.0
set()


3

In [45]:

manual_outlier_df.to_csv('new_data/complete-data.csv', index=False)
manual_outlier_df.sort_values('year')
# manual_outlier_df.to_csv('output/loan_dev_transformed_without_outliers.csv', index=False)

,playerID,year,tmID,GP,oRebounds,dRebounds,dq,ft%,fg%,three%,...,award_count,career_year,playoff_progression_rolling,playoff_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,rpg_rolling,ppg_rolling,stats_post_rolling
691,397,3,573,24.0,1.38,1.83,0.00,0.61,0.41,0.00,...,0.0,3,0.5,0.5,-112.170,0.0,24.65,2.605,5.545,-0.285
287,191,3,568,3.0,0.33,0.33,0.00,0.75,0.50,0.00,...,0.0,3,1.5,1.0,-100.605,0.0,30.65,3.235,6.050,-0.370
97,70,3,559,32.0,1.44,4.38,0.00,0.83,0.46,0.40,...,0.0,3,0.5,0.5,-212.575,0.0,79.70,6.470,9.690,0.215
284,187,3,573,32.0,0.50,1.50,0.00,0.76,0.38,0.32,...,0.0,3,0.0,0.0,-171.655,0.0,22.05,2.215,7.270,0.000
900,504,3,558,32.0,0.97,3.98,0.05,1.70,0.71,0.83,...,0.0,3,2.5,1.0,-101.850,0.0,33.90,2.225,3.500,-0.115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,247,11,556,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,9,0.5,0.5,-267.790,0.0,53.75,6.000,16.850,-0.135
424,260,11,574,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,6,1.0,0.5,-228.340,0.0,54.55,5.355,9.310,-0.205
427,262,11,745,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,5,0.5,0.5,-131.025,0.0,20.25,2.445,4.365,-0.215
836,466,11,573,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,9,4.0,1.0,-173.405,0.0,46.45,4.360,14.360,-1.135
